## 1. 그레이디언트 소실과 폭주 문제
### 1) 글로럿과 He 초기화

In [1]:
from tensorflow import keras

In [1]:
# He 초기화

keras.layers.Dense(10, activation='relu', kernel_initializer='he_normal')

In [ ]:
# He 초기화 (fan_out 기반)

he_avg_init = keras.initializers.VarianceScaling(scale=2., mode='fan_avg',
                                                distribution='uniform')
keras.layers.Dense(10, activation='relu', kernel_initializer='he_normal')

### 2) 수렴하지 않는 활성화 함수
- 은닉층에서 주로 쓰이는 ReLU 함수는 완벽하지 X (죽은 ReLU)
- 일반적으로 SELU > ELU > LeakyReLU(그리고 변종들) > ReLU > tanh > 로지스틱순으로 적용 (문제에 따라 적절한 함수사용하기)

##### LeakyReLU 활성화 함수
- LeakyReLU 층을 만들고 모델에서 적용하려는 층 뒤에 추가하기
- 가중치 초기화 방법 : He

In [ ]:
# 코드만 보기 (실행 X)

model = keras.model.Sequential([
    # [...]
    keras.layers.Dense(10, kernel_initializer='he_normal')
    keras.layers.LeakyReLU(alpha=0.2),
    # [...]
])

##### SeLU 활성화 함수
- activation의 옵션으로
- 가중치 초기화 방법 : 르쿤(lecun)

In [4]:
layer = keras.layers.Dense(10, activation='selu',
                          kernel_initializer='lecun_normal')

### 3) 배치 정규화
- 입력을 정규화한 다음, 두 개의 파라미터(스케일 조정과 이동)로 스케일 조정하기($\gamma z+\beta$)
- why? 입력을 정규화하면 대부분 0에 가까운 값이 됨. sigmoid 함수는 0 부근에서 선형구간에 빠지므로 이를 해결하기 위해 $\gamma$와 $\beta$를 적용
- 추가적으로 테스트셋에서 활용할 $\mu$와 $\sigma$ 파라미터를 학습함 (테스트의 입력도 똑같이 해줘야 하므로)
- 
- 은닉층의 활성화 함수 전이나 후에 배치 정규화 층을 넣어주기

In [2]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.BatchNormalization(),           
    keras.layers.Dense(300, activation='elu', kernel_initializer='he_normal'),
    keras.layers.BatchNormalization(),    # 활성화 함수 후에 배치 정규화 층을 넣음
    keras.layers.Dense(100, activation='elu', kernel_initializer='he_normal'),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(10, activation='softmax')
])

In [5]:
model.summary()   # 파라미터가 4개 (gamma, beta, mu, sigma)
# 이때, mu와 sigma는 역전파로 학습되지 않기 때문에 Non-trainable 파라미터(지수이동 평균으로 학습됨)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
batch_normalization (BatchNo (None, 784)               3136      
_________________________________________________________________
dense (Dense)                (None, 300)               235500    
_________________________________________________________________
batch_normalization_1 (Batch (None, 300)               1200      
_________________________________________________________________
dense_1 (Dense)              (None, 100)               30100     
_________________________________________________________________
batch_normalization_2 (Batch (None, 100)               400       
_________________________________________________________________
dense_2 (Dense)              (None, 10)                1

In [6]:
# trainable 여부

[(var.name, var.trainable) for var in model.layers[1].variables]

[('batch_normalization/gamma:0', True),
 ('batch_normalization/beta:0', True),
 ('batch_normalization/moving_mean:0', False),
 ('batch_normalization/moving_variance:0', False)]

In [8]:
# 배치 정규화 층을 만들 때, 훈련동안 매 반복마다 케라스에서 호출될 두 개의 연산이 
# 함께 생성됨 (두 개의 연산이 뭐자)

model.layers[1].updates

[<tf.Operation 'cond/Identity' type=Identity>,
 <tf.Operation 'cond_1/Identity' type=Identity>]

##### 활성화 함수 전에 배치 정규화 층 넣기
- 은닉층에서 활성화 함수를 지정 X -> 배치 정규화 층 뒤에 별도의 층으로 추가해주기

In [ ]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.BatchNormalization(),           
    keras.layers.Dense(300, kernel_initializer='he_normal'),
    keras.layers.BatchNormalization(),  # 활성화 함수 전에 넣어줌
    keras.layers.Activation('elu'),
    keras.layers.Dense(100, activation='elu', kernel_initializer='he_normal'),
    keras.layers.BatchNormalization(), 
    keras.layers.Activation('elu'),
    keras.layers.Dense(10, activation='softmax')
])

### 4) 그레디언트 클리핑
- 역전파될 때 일정 임곗값을 넘어서지 못하게 그레디언트를 잘라내는 것 (인기있는 방법)
- 순환신경망에서는 배치 정규화를 적용하기 어려워 이 방법을 많이 사용함

In [ ]:
optimizer = keras.optimizers.SGD(clipvalue=1.0)
model.compile(loss='mse', optimizer=optimizer)

- clipvalue=1.0 : 그레디언트 벡터의 모든 원소를 -1.0과 1.0 사이로 클리핑<br/>ex) [0.9, 100.0] -> [0.9, 1.0]
- 방향을 바꾸지 않도록 -> clipnorm 지정

## 2. 사전훈련된 층 재사용하기
- __전이 학습__ : 재사용하는 층을 모두 동결 -> 모델을 훈련하고 성능 평가 -> 맨 위에 있는 한두 개의 층 동결해제하고 확인 (학습률 줄이기)
- __비지도 사전 훈련__ : (비지도적으로) 훈련된 오토인코더나 GAN 판별자의 하위층을 재사용하고 출력층 추가 -> 지도 학습으로 최종 네트워크를 세밀하게 튜닝
- __보조 작업에서의 사전 훈련__ : (레이블된 훈련데이터가 적다면) 훈련 데이터를 쉽게 얻을 수 있는 보조 작업에서의 신경망 모델을 훈련 -> 훈련된 신경망의 하위층을 재사용<br/>

	cf) 자기 지도 학습(비지도 학습) : 데이터에서 스스로 레이블을 생성하여 지도 학습 기법을 이용 

## 3. 고속 옵티마이저
### 1) 모멘텀 최적화
- 모멘텀 : 운동량이나 가속도
- 그레디언트를 속도가 아니라 __가속도__로 사용, 이전 그레디언트 값을 중요하게 생각!


1. $m \leftarrow \beta m - \eta \nabla_\theta J(\theta)$
2. $\theta \leftarrow \theta + m$

In [2]:
optimizer = keras.optimizers.SGD(lr=0.001, momentum=0.9) # 모멘텀 0.9에서 보통 잘 작동

### 2) 네스테로프 가속 경사
- 모멘텀 최적화의 한 변종
- 현재 위치 θ가 아니라, 모멘텀 방향으로 조금 앞선 θ+βm에서 그레디언트를 계산 (좀 더 정확)

In [3]:
optimizer = keras.optimizers.SGD(lr=0.001, momentum=0.9, nesterov=True)

### 3) AdaGrad
- (한쪽이 길쭉한 그릇 문제에서) 경사 하강법은 가장 가파른 경사를 따라 내려감<br/>
   -> 이를 일찍 감지하고 전역 최적점 쪽으로 좀 더 정확한 방향 잡으면 좋을 듯 => AdaGrad
- 적응적 학습률 : 학습률을 감소시키는 데, 경사가 완만한 차원보다 가파른 차원에 대해 더 빠르게 감소시킴 (많이 변화한 변수는 최적값에 근접했을 것이라는 가정에 의해)

### 4) RMSProp
- AdaGrad가 너무 빨리 느려져서 수렴하지 못하는 위험 존재<br/>
=> (시작부터의 모든 그레디언트가 아닌) 가장 최근 반복에서 비롯된 그레디언트만 누적

In [ ]:
optimizer = keras.optimizers.RMSprop(lr=0.001, rho=0.9) # rho는 beta

In [7]:
# l2 규제

layer = keras.layers.Dense(100, activation='elu',
                          kernel_initializer='he_normal',
                          kernel_regularizer=keras.regularizers.l2(0.01))

In [6]:
# 모든 층에 동일한 매개변수 이용하는 경우 많음 -> partial 함수 이용하기

from functools import partial

RegularizedDense = partial(keras.layers.Dense,
                          activation='elu',
                          kernel_initializer='he_normal',
                          kernel_regularizer=keras.regularizers.l2(0.01))

model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    RegularizedDense(300),
    RegularizedDense(100),
    RegularizedDense(10, activation='softmax',
                     kernel_initializer='glorot_uniform')
])

In [ ]:
# 드롭아웃
model = keras.models.Sequential([ 
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dropout(rate=0.2), 
    keras.layers.Dense(300, activation="elu", kernel_initializer="he_normal"),
    keras.layers.Dropout(rate=0.2),
    keras.layers.Dense(100, activation="elu", kernel_initializer="he_normal"),
    keras.layers.Dropout(rate=0.2),
    keras.layers.Dense(10, activation="softmax")
])

In [9]:
import numpy as np

In [10]:
# MC 드롭아웃
y_probas = np.stack([model(X_test_scaled, training=True)
                     for sample in range (100)])
y_proba = y_probas.mean(axis=0)

NameError: name 'X_test_scaled' is not defined

In [11]:
keras.layers.Dense(100, activation='elu', kernel_initializer='he_normal',
                  kernel_constraint=keras.constraints.max_norm(1.))